In [ ]:
import pyomo.environ as pyo

from scipy.spatial.distance import pdist, squareform

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

import time
from itertools import cycle

#from pyomo.contrib.appsi.solvers import Highs

import pandas as pd
from decimal import Decimal

In [ ]:
# FILE = 'test1'
# ACCURATE = False
# FILE = 'shuffled_ordered_data/R211_shuffled_ordered'
FILE = 'shuffled_data/RC201_shuffled'
# FILE = 'test_data/test6'
# FILE = 'test_data/R211_test'
# FILE = 'experiment2_data/C201_test'
FILE_NAME = "C:/Users/caleb/Desktop/Study/MA4079-Final_Year_Project/Data/"+FILE+".csv"
data = pd.read_csv(FILE_NAME)
data.columns = data.columns.astype(str)

In [ ]:
PLANNING_TIME = 0
CUSTOMERS_LIST = []
data = data.loc[(data['DUE DATE'] >= PLANNING_TIME) | (data['CUST NO.'].isin(CUSTOMERS_LIST))]
data.shape

In [ ]:
N = 100
CAPACITY = 50
N_VEHICLES = 3
R = 3 #number of trips. set to a fiarly large number
DEMAND_TYPES = 5
VEHICLE_COMPARMENTS = 5

vehicle_capacities = np.array([[CAPACITY]*DEMAND_TYPES]*N_VEHICLES)
df = data[:N+1]
cust_no = np.array(df.loc[:,'CUST NO.'])
demands = np.array(df.loc[:,'DEMAND'])
service_time = np.array(df.loc[:,'SERVICE TIME'])#demands #np.zeros(len(demands)) #demands 
demands_zeros = np.zeros((len(demands), DEMAND_TYPES-1))
demands = np.hstack((demands.reshape(N+1,1),demands_zeros))
coordinates = np.array(df.loc[:,['XCOORD','YCOORD']])
start_time_windows = np.array(df.loc[:,'READY TIME'])
end_time_windows = np.array(df.loc[:,'DUE DATE']) 

demands_index = [i for i in range(1,N+1)]

START_TIME = start_time_windows[0]
END_TIME = end_time_windows[0]

print(START_TIME)
print(END_TIME)

In [ ]:
coordinates

In [ ]:
end_time_windows

In [ ]:
demands_shuffled = np.array(df.iloc[:,4:9])
demands_shuffled

In [ ]:
TOTAL_GOODS = demands_shuffled.sum()
TOTAL_GOODS

In [ ]:
print('Max possible score:', np.abs(demands_shuffled).sum())
#print('Max goods:', np.abs(demands).sum()/2)
print ('Veh max Capacity:', vehicle_capacities.sum())

In [ ]:
distances = squareform(pdist(coordinates, metric="euclidean"))
def trunc(values, decs=1):
    return np.trunc(values*10**decs)/(10**decs)
distances = trunc(distances)
# distances = np.round(distances, decimals=2)

#travel_time = distances/5
travel_time = distances
# travel_time

In [ ]:
# Choosing colors
cmap = mpl.colormaps["Dark2"]
colors = cycle(cmap.colors)

# Now the figure
fig, ax = plt.subplots(figsize=[6, 5], dpi=100)

for i, coord in enumerate(coordinates):
    x = coord[0]
    y= coord[1]
    ax.scatter(x,y)
    ax.annotate(i, (x,y))

In [ ]:
model = pyo.ConcreteModel()

## Sets

In [ ]:
model.D = pyo.Set(initialize=demands_index) #set of vessels
model.N = pyo.Set(initialize=range(len(demands))) # set of vessels+depot
model.N1 = pyo.Set(initialize=range(len(demands)+1)) # set of vessels+depot+end
model.A = pyo.Set(initialize=[(i, j) for i in model.N for j in model.N if i != j]) #set of arcs
model.K = pyo.Set(initialize=range(N_VEHICLES)) #set of vehicles
model.R  = pyo.Set(initialize=range(R)) # set of trips
model.F = pyo.Set(initialize = range(DEMAND_TYPES)) #set of demand types

## Parameters

In [ ]:
zeta = 0.1 # travel cost per distance factor
price = [1]*DEMAND_TYPES

In [ ]:
model.q = pyo.Param(model.K, model.F, initialize={(i,f): vehicle_capacities[i,f] for i in model.K for f in model.F}) #capacity of vehicles
model.c = pyo.Param(model.A, initialize={(i, j): zeta*distances[i, j] for (i, j) in model.A}) #cost of travel per arc
model.n = pyo.Param(model.N, model.F, initialize={(i,f): demands_shuffled[i,f] for i in model.N for f in model.F}) #loading of all nodes
model.r = pyo.Param(model.N, model.F, initialize={(i,f): price[f]*demands_shuffled[i,f] for i in model.N for f in model.F}) #revenue of all nodes
model.s = pyo.Param(model.N, initialize=service_time) #service time per node
model.t = pyo.Param(model.N, model.N, initialize={(i, j): travel_time[i, j] for i in model.N for j in model.N}) #travel time per arc)
model.a = pyo.Param(model.N, initialize=start_time_windows) #start time windows per node
model.b = pyo.Param(model.N, initialize=end_time_windows) #end time windows per node

## Variables

In [ ]:
new_start_time_windows = np.append(start_time_windows, START_TIME)
new_start_time_windows = [[new_start_time_windows]*R]*N_VEHICLES

new_end_time_windows = np.append(end_time_windows, END_TIME)
new_end_time_windows = [[new_end_time_windows]*R]*N_VEHICLES

In [ ]:
def time_windows(model, i,k,r):
    return (new_start_time_windows[k][r][i], new_end_time_windows[k][r][i])

## Variables

In [ ]:
model.X = pyo.Var(model.A, model.K, model.R, within=pyo.Binary) # decision to move along arc ij by vehicle K on trip R
model.Y = pyo.Var(model.N, model.K, model.R, within=pyo.Binary) # decision of if node N is visited by vehicle K on trip R
model.T = pyo.Var(model.N1, model.K, model.R, within=pyo.NonNegativeReals, bounds= time_windows) # visit time at node N by vehicle K on trip R
model.S = pyo.Var(model.K, model.R, within=pyo.NonNegativeReals, bounds= (0,CAPACITY*DEMAND_TYPES)) #serivce time at the depot for the start of each trip
model.L = pyo.Var([0,N+1], model.K, model.R, model.F, within=pyo.NonNegativeReals, bounds= (0,CAPACITY)) # fuel load on vehicle K on trip R for fuel type F
model.U = pyo.Var(model.N, model.K, model.R, model.F, within=pyo.NonNegativeReals, bounds= (0,CAPACITY))

## Constraints

In [ ]:
M = np.zeros((N+1,N+1))
beta = 0.2 # rate of fuel transfer at the depot
for i in model.N:
    for j in model.N:
         M[i,j] = model.b[i] + model.s[i] + model.t[i,j] - model.a[j]
for j in model.N:
      M[0,j] = model.b[0] + CAPACITY*DEMAND_TYPES*beta + model.t[0,j] - model.a[j]
print(np.max(M))


In [ ]:
def visit_once_rule(model,i): #2/13
    return sum(model.Y[i,k,r] for k in model.K for r in model.R) <= 1
model.visit_once_rule = pyo.Constraint(model.D, rule=visit_once_rule)

def arcs_out_rule(model, i, k,r): #3/14 modifyied Arcs out rule
    return sum(model.X[i, j, k,r] for j in model.N if i != j) == model.Y[i,k,r]
model.arcs_out_rule = pyo.Constraint(model.N, model.K, model.R, rule=arcs_out_rule)

def arcs_in_rule(model, i, k,r): #3/14 modifyied Arcs in rule
    return sum(model.X[j, i, k,r] for j in model.N if i != j) == model.Y[i,k,r]
model.arcs_in_rule = pyo.Constraint(model.N, model.K, model.R, rule=arcs_in_rule)

def feasbile_time_along_arc(model,i,j, k,r): #71
    if i == j:
        return pyo.Constraint.Skip
    else:
        # return model.T[i,k,r] + model.s[i] + model.t[i,j] <= model.T[j,k,r] + M*(1-model.X[i,j,k,r])
        return model.T[i,k,r] + model.s[i] + model.t[i,j] <= model.T[j,k,r] + M[i,j]*(1-model.X[i,j,k,r])
model.feasbile_time_along_arc = pyo.Constraint(model.D, model.D, model.K, model.R, rule=feasbile_time_along_arc)

def feasbile_start_time (model,j,k,r): #72 
    # return model.T[0,k,r] + model.S[k,r] + model.t[0,j] <= model.T[j,k,r] + M*(1-model.X[0,j,k,r])
    return model.T[0,k,r] + model.S[k,r] + model.t[0,j] <= model.T[j,k,r] + M[0,j]*(1-model.X[0,j,k,r])
model.feasbile_start_time = pyo.Constraint(model.D, model.K, model.R, rule=feasbile_start_time)

def feasbile_end_time (model,i,k,r): #66
    # return model.T[i,k,r] + model.s[i] + model.t[i,0] <= model.T[N+1,k,r] + M*(1-model.X[i,0,k,r])
    return model.T[i,k,r] + model.s[i] + model.t[i,0] <= model.T[N+1,k,r] + M[i,0]*(1-model.X[i,0,k,r])
model.feasbile_end_time = pyo.Constraint(model.D, model.K, model.R, rule=feasbile_end_time)

def trip_end_before_start (model, k,r): #67
    if r == model.R.last():
        return pyo.Constraint.Skip
    else:
        return model.T[N+1,k,r] <= model.T[0,k,r+1]
model.trip_end_before_start = pyo.Constraint(model.K, model.R, rule=trip_end_before_start)    

def trip_end_after_start (model, k,r): #modified (necessary)
    return model.T[0,k,r] + model.S[k,r] <= model.T[N+1,k,r]
model.trip_end_after_start = pyo.Constraint(model.K, model.R, rule=trip_end_after_start)    


## Service time at depot

In [ ]:
full_capacity = [CAPACITY]*VEHICLE_COMPARMENTS

def service_time_at_depot (model,k,r): #73 modified
    if r == model.R.first():
        return model.S[k,r] == 0 #beta * (sum(full_capacity)- sum(model.L_initial[k,f] for f in model.F))
    else:
        return model.S[k,r] == beta* (sum(full_capacity) - sum(model.L[N+1,k,r-1,f] for f in model.F))
model.service_time_at_depot = pyo.Constraint(model.K, model.R, rule=service_time_at_depot)

## Vehicle Loading

In [ ]:

def fuel_del_to_visit(model,i,k,r,f): #if cust i is visited, U>0 else set  U = 0
    return model.U[i,k,r,f] == model.Y[i,k,r] * model.n[i,f]

    #  return model.U[i,k,r,f] <= model.Y[i,k,r] * CAPACITY
model.fuel_del_to_visit = pyo.Constraint(model.N, model.K, model.R, model.F,rule=fuel_del_to_visit)

def fuel_load_in_veh_at_start(model,k,r,f): #10
    if r == model.R.first():
        return pyo.Constraint.Skip #loading for first trip would be handled later
    else:
        # return pyo.Constraint.Skip
        return model.L[0,k,r,f] == full_capacity[f]
model.fuel_load_in_veh_at_start = pyo.Constraint(model.K,model.R, model.F,rule=fuel_load_in_veh_at_start)

def load_in_veh_at_end(model,k,r,f): #fuel load at end same as or exceeds the load for the trip
    return model.L[N+1,k,r,f] == model.L[0,k,r,f] - sum(model.U[i,k,r,f] for i in model.N) 
model.load_in_veh_at_end = pyo.Constraint(model.K,model.R, model.F,rule=load_in_veh_at_end)


## Objective

In [ ]:
'''
#obj to minise travel cost
model.obj = pyo.Objective(
    expr=sum(
        model.X[i, j, k,r] * model.t[i,j]
        for (i, j) in model.A
        for k in model.K
        for r in model.R
        ), 
    sense=pyo.minimize,
)
'''
'''
#obj without travel costs
model.obj = pyo.Objective(
    expr=sum(
        model.Y[i, k,r] * np.abs(model.n[i,f])
        for i in model.D
        for k in model.K
        for r in model.R
        for f in model.F
        ), 
    sense=pyo.maximize,
)
'''

#obj with travel costs
model.obj = pyo.Objective(
    expr=sum(
        model.Y[i, k,r] * np.abs(model.r[i,f])
        for i in model.D
        for k in model.K
        for r in model.R
        for f in model.F
    ) - sum(model.X[i, j, k,r] * model.c[i,j]
        for (i, j) in model.A
        for k in model.K
        for r in model.R),
    sense=pyo.maximize,
)


## Inital Conditions

In [ ]:
def route_pair(route):
    route_pair = []
    for i in range(len(route)-1):
        from_node = route[i]
        to_node = route[i+1]
        if from_node == to_node:
            raise Exception('i == j')
        else:
            route_pair.append((from_node,to_node))
    return route_pair

In [ ]:
full_capacity = [CAPACITY]*VEHICLE_COMPARMENTS
empty_capacity = [0]*VEHICLE_COMPARMENTS

route1_cust_no = [1]
route2_cust_no = [1]
route3_cust_no = [1]

depot_index = np.where(cust_no == 1)[0][0]
if len(route1_cust_no) != 0:
    route1_index = np.where(cust_no == route1_cust_no)[0][0]
else:
    route1_index = None
if len(route2_cust_no) != 0:
    route2_index = np.where(cust_no == route2_cust_no)[0][0]
else:
    route2_index = None
if len(route3_cust_no) != 0:
    route3_index = np.where(cust_no == route3_cust_no)[0][0]
else:
    route3_index = None

#time start at depot
time_start_1 = 0#50 - travel_time[depot_index,route1_index]
time_start_2 = 0#50 - travel_time[depot_index,route2_index]
time_start_3 = 0#50 - travel_time[depot_index,route3_index]

#loading of vehicle before top up
loading1 = full_capacity
loading2 = full_capacity
loading3 = full_capacity
#check that given load does not exceed vehicle capacity
for i, cap in enumerate(full_capacity):
    if (loading1[i] > cap) or (loading2[i] > cap) or (loading3[i] > cap) :
        raise Exception('Capacity over limit')
    

routes_initial = [route1_cust_no, route2_cust_no, route3_cust_no]
time_start_initial = [time_start_1,time_start_2,time_start_3]
loading_initial = [loading1,loading2,loading3]

In [ ]:
def trip_split(route):
    trip =[]
    temp = [0]
    for i in range(1, len(route)):
        temp.append(route[i])
        if route[i] == 0:
            trip.append(temp)
            temp = [0]
    return trip

In [ ]:
print ("Time start 1:", time_start_1)
print ("Time start 2:", time_start_2)
print ("Time start 3:", time_start_3)

In [ ]:
def inital_loading(model,k,f):
    return model.L[0,k,0,f] == loading_initial[k][f]

model.inital_loading = pyo.Constraint(model.K,model.F,rule=inital_loading)

if route1_index != 0:
    def inital_route_1(model):
        return model.X[0,route1_index,0,0] == 1
    model.inital_route_1 = pyo.Constraint(rule=inital_route_1)
if route2_index != 0:
    def inital_route_2(model):
        return model.X[0,route2_index,1,0] == 1
    model.inital_route_2 = pyo.Constraint(rule=inital_route_2)
if route3_index != 0:
    def inital_route_3(model):
        return model.X[0,route3_index,2,0] == 1
    model.inital_route_3 = pyo.Constraint(rule=inital_route_3)

def initial_time_start_at_depot (model,k):
    return model.T[0,k,0] == time_start_initial[k]
model.initial_time_start_at_depot = pyo.Constraint(model.K, rule=initial_time_start_at_depot)


## Reducing arcs

In [ ]:
arcs_index = [(i,j) for i in range(len(demands)) for j in range(len(demands)) if i!=j]
len(arcs_index)

In [ ]:
count = 0
for arc in model.A:
    i,j = arc[0],arc[1]
    if start_time_windows[i] +service_time[i] +travel_time[i,j] > end_time_windows[j]:
        for k in model.K:
            for r in model.R:
                model.X[i,j,k,r].fix(0) #remove these arcs
                count += 1
count
# len(arcs_index)

## Model Check

def cust_no_index(cust_no:int, cust_no_list):
    '''
    converts customer_number to index in data
    '''
    temp = np.where(cust_no_list == cust_no)
    #print (len(temp[0]))
    if len(temp[0]) ==1:
        return temp[0][0]
    else:
        return -1

def trip_split(route):
    trip =[]
    temp = [route[0]]
    for i in range(1, len(route)):
        temp.append(route[i])
        if route[i] == 0:
            trip.append(temp)
            temp = [0]
    return trip

routes_custno =  [[1, 73, 96, 84, 70, 99, 16, 53, 24, 20, 23, 87, 1, 57, 98, 14, 18, 75, 25, 90, 94, 81, 1], [1, 93, 40, 37, 32, 29, 30, 39, 45, 9, 8, 80, 10, 1, 97, 69, 56, 5, 2, 71, 101, 1], [1, 66, 60, 46, 6, 3, 13, 17, 12, 89, 1, 95, 85, 51, 35, 27, 33, 55, 92, 49, 26, 78, 59, 1]]
routes = []
for route in routes_custno:
    routes.append([cust_no_index(i,cust_no) for i in route])
# routes =  [[0, 23, 6, 13, 0, 12, 1, 0], [0, 11, 18, 2, 5, 3, 4, 8, 0]]
node_list= []
for k, route in enumerate(routes):
    route = trip_split(route)
    route = [i for i in route if len(i) >2]
    print(route)
    for r, trip in enumerate(route[::-1]):
        prev_node = trip[0]
        for node in trip[1:]:
            # model.X[prev_node,node,k,(R-r-1)] = 1
            # model.Y[node,k,(R-r-1)] = 1
            model.X[prev_node,node,k,(R-r-1)].fix(1)
            model.Y[node,k,(R-r-1)].fix(1)
            prev_node = node


## Solve

In [ ]:
TIME_LIMIT = 60
THREADS = 1
MIPFocus = 0 #gurobi, default is 0, set to 1 to focus more on finding feasible solutions

SOVLER_ENGINE = 'gurobi'
#solvers glpk  appsi_highs cplex gurobi

solver = pyo.SolverFactory(SOVLER_ENGINE)

if SOVLER_ENGINE == 'cbc':
        solver.options['seconds'] = TIME_LIMIT
elif SOVLER_ENGINE == 'glpk':
        solver.options['tmlim'] = TIME_LIMIT
elif SOVLER_ENGINE == 'appsi_highs':
        solver.options['time_limit'] = TIME_LIMIT
        #solver.options['parallel'] = True
        solver.options['threads'] = THREADS
elif SOVLER_ENGINE == 'cplex':
        solver.options['timelimit'] = TIME_LIMIT
        solver.options['threads'] = THREADS
elif SOVLER_ENGINE == 'gurobi':
        solver.options['timelimit'] = TIME_LIMIT
        solver.options['threads'] = THREADS
        solver.options['MIPFocus'] = MIPFocus

# sol = solver.solve(tee= True)
sol = solver.solve(model, tee= True, warmstart=True) #, warmstart=True , logfile= 'log.txt'
# logfile = FILE+str(N)+'.txt'
# sol = solver.solve(model, tee= True, logfile= logfile) #, warmstart=True , logfile= 'log.txt'

In [ ]:
print(sol.solver.status)
print(sol.solver.termination_condition)
print (model.obj())

print ('Total amount of goods delivered:',sum(
        model.X[i, j, k,r].value * np.abs(model.n[i,f])
        for (i, j) in model.A
        for k in model.K
        for r in model.R
        for f in model.F
        ) )

print ('Total number of vessels visited:', sum(model.Y[i,k,r].value
                                               for i in model.D
                                               for k in model.K
                                               for r in model.R))

print ('Total travel time:', sum(model.X[i,j,k,r].value * model.t[i,j]
                                               for (i,j) in model.A
                                               for k in model.K
                                               for r in model.R))

In [ ]:
# model.display()

## Plot

In [ ]:
# Choosing colors
colors = ["red","blue","green",'orange','purple','yellow','black','black']

fig, ax = plt.subplots(figsize=[6, 5], dpi=500)
# ax.set_aspect('equal')

# Now the figure
for i, coord in enumerate(coordinates):
    x = coord[0]
    y= coord[1]
    ax.scatter(x,y, color = 'grey')
    ax.annotate(cust_no[i], (x,y))

#print ('Number of nodes:',N)
for k in range(N_VEHICLES):
    c = colors[k]
    for r in model.R:
        for i in range(len(demands)):
            for j in range(len(demands)):
                if i ==j: 
                    continue
                elif np.isclose(model.X[i, j, k,r].value, 1, atol=1e-1):
                    coord1 = coordinates[i]
                    coord2 = coordinates[j]
                    x1 = coord1[0]
                    y1 = coord1[1]
                    x2 = coord2[0]
                    y2 = coord2[1]
                    if i ==0 :
                        f = 0
                        d = 0
                    else: 
                        demand_shuffled = demands_shuffled[i]
                        f= np.nonzero(demand_shuffled)
                        d = demand_shuffled[f].astype(int)[0]
                        f = f[0][0]

                    ax.scatter(x1,y1, color = c)

                    # ax.annotate((i,d), (x1,y1))
                    ax.plot((x1,x2), (y1,y2), color=c, label= k+1)
#ax.legend()

for k in range(N_VEHICLES):
    print("\nVehicle no:", k+1)
    node = 0
    #print (node, end ='')
    for r in model.R:
        for i in range(len(demands)):
            for j in range(len(demands)):
                    if node == j:
                        continue
                    elif np.isclose(model.X[node, j, k,r].value, 1, atol=1e-1):
                        print (cust_no[node], " --", cust_no[j], end =' ')
                        print ("R:",r)
                        node = j
                        break
            if node == 0:
                break
    #print ("\n")

## Checking loading constraints

In [ ]:
for k in range(N_VEHICLES):
    print('\n',"Vehicle no:", k+1)
    print("Max Capacity of vehicle:", vehicle_capacities[k])
    node = 0
    #print (node, end ='')
    for r in model.R:
        print ('\n', 'Trip:', r)
        #for f in model.F:
        print ('Total loaded onto vehicle', r,':',model.S[k,r].value)
        load = [model.L[0,k,r,f].value for f in model.F]
        print ('Vehicle Load at depot', [model.L[0,k,r,f].value for f in model.F])
        total = np.zeros(DEMAND_TYPES)
        for i in range(len(demands)):
            for j in range(len(demands)):
                    if node == j:
                        continue
                    elif np.isclose(model.X[node, j, k,r].value, 1, atol=1e-1):
                        
                        print ('Cust is:', cust_no[node])
                    
                        print ('Vehicle load', load)
                        print ('Load at node:', demands_shuffled[node])
                        load -= demands_shuffled[node]
                        total += demands_shuffled[node]
                        node = j
                        break
            if node ==0:
                 break
        print ('Remaining Vehicle load', [model.L[N+1,k,r,f].value for f in model.F])
        print('Total delviered:',total)

## Checking time constraints

In [ ]:
for k in range(N_VEHICLES):
    print("Vehicle no:", k+1)
    node = 0
    for r in model.R:
        print ('\n', 'Trip no:', r)
        print ('Total load of trip', r,':',model.S[k,r].value)
        for i in range(len(demands)):
            for j in range(len(demands)):
                    if node == j:
                        continue
                    elif np.isclose(model.X[node, j, k,r].value, 1, atol=1e-1):
                        print ('Time window at cust:',cust_no[node],': [', model.a[node], model.b[node], ']')
                        print ('Time reach cust:',cust_no[node],':', model.T[node,k,r].value, '\n')
                        print ('Service time:', model.s[node])
                        print ('Travel time from', cust_no[node],'to', cust_no[j] ,':', model.t[node,j], '\n')
                        node = j
                        break
            if node ==0:
                 break
        print ('Time at end,',r,':', model.T[N+1,k,r].value)


## Schedules per vessel

In [ ]:
fig, ax = plt.subplots(figsize = (5,20))
ax.set_xlim(0, N_VEHICLES + 2)
ax.set_ylim(START_TIME, END_TIME)
ax.set_xticks(np.arange(1, N_VEHICLES + 1))
for k in range(N_VEHICLES):
    ax.plot([k+1, k+1], [START_TIME,END_TIME], linewidth=2, color='b')  #vessel docking period
    for r in range(R): #range(R):
        for i in range(len(demands)):
            for j in range(len(demands)):
                if i ==j: 
                    continue
                elif np.isclose(model.X[i, j, k,r].value, 1, atol=1e-1):
                    
                    time_start_at_node = model.T[i,k,r].value
                    if i !=0: #servicing customers
                        service_time_at_node = model.s[i]
                        ax.add_patch(plt.Rectangle((k+1-0.25, time_start_at_node), 0.5, service_time_at_node, edgecolor='black',\
                                        linewidth=2,facecolor='none'))
                        ax.text(k+1-0.125, time_start_at_node+(service_time_at_node/2), cust_no[i], color = 'black', fontsize = 8)
                    else: #starting at terminal
                        service_time_at_node = model.S[k,r].value
                        ax.add_patch(plt.Rectangle((k+1-0.25, time_start_at_node), 0.5, service_time_at_node, edgecolor='red',\
                                        linewidth=2,facecolor='none'))
                    travelling_time = model.t[i,j]
                    ax.add_patch(plt.Rectangle((k+1-0.25, time_start_at_node+service_time_at_node), 0.5, travelling_time, edgecolor='orange',\
                                        linewidth=2,facecolor='none'))

for k in range(N_VEHICLES):
    for r in range(R):
        time_start_at_node = model.T[0,k,r].value
        service_time_at_node = model.S[k,r].value
        ax.add_patch(plt.Rectangle((k+1-0.25, time_start_at_node), 0.5, service_time_at_node, edgecolor='red',\
                        linewidth=2,facecolor='none'))
                        
